In [1]:
import time
import json
import os
from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen
from collections import defaultdict
import re
import pandas as pd
import numpy as np

CORPUS = pd.read_csv("data/final_corpus.csv", encoding="utf-8").set_index("id")
METADATA = pd.read_csv(
    "data/lang-8-users.csv",
    encoding="utf-8",
    dtype={"user_id": str}
)

In [2]:
LOCATION_MAPS = {
    0: "Japan", 
    1: "China",
    2: "Taiwan",
    3: "Vietnam",
    4: "Russia",
    5: "Korea",
    6: "Brazil",
    7: "U.S.A",
    8: "France",
    9: "Spain"
}

OCCUPATION_MAPS = {
    0: "Artist",
    1: "Designer",
    2: "Engineer",
    3: "Housewife/ Househusband",
    4: "Office worker",
    5: "Programmer",
    6: "Student",
    7: "Teacher",
    8: "Other"
}

GENDER_MAPS = {
    0: "Female",
    1: "Male"
}

SENT_TYPE_MAPS = {
    0: "original",
    1: "corrected"
}

METADATA_KEYS = {"sex", "occupation", "location", "Lpoints"}

In [3]:
def construct_metadata_dict(param_dict):
    """Construct the metadata dictionary based on param_dict provided"""
    new_dict = {}
    
    for k, v in param_dict.items():
        if k in METADATA_KEYS:
            if v == -1:
                continue
            if k == "sex":
                if v in GENDER_MAPS:
                    v = GENDER_MAPS[v]
            elif k == "occupation":
                if v in OCCUPATION_MAPS:
                    v = OCCUPATION_MAPS[v]
            elif k == "location":
                if v in LOCATION_MAPS:
                    v = LOCATION_MAPS[v]
            new_dict[k] = v 
    return new_dict

In [4]:
def get_by_range(corpus, sent_type, num_range=(10, 15)):
    """Returns sentence pairs with only sentence length within num_range, sent_type 
    is used to specify whether the restrictions are put on original or corrected sentences
    """
    lower, upper = num_range
    if sent_type in SENT_TYPE_MAPS:
        mask = (corpus[SENT_TYPE_MAPS[sent_type]].str.len() >= lower) & (
            corpus[SENT_TYPE_MAPS[sent_type]].str.len() <= upper
        )
        return corpus.loc[mask]
    else:
        org_mask = (corpus["original"].str.len() >= lower) & (
            corpus["original"].str.len() <= upper
        )
        cor_mask = (corpus["corrected"].str.len() >= lower) & (
            corpus["corrected"].str.len() <= upper
        )
        mask = org_mask & cor_mask
        return corpus.loc[mask]

In [5]:
def get_by_multiple_corrections(corpus, n_corrections):
    """Return the sentences which only contains n number of corrections"""
    columns = CORPUS.columns.to_list()
    grouped = corpus.groupby("original").agg("count").reset_index()
    filtered = grouped[grouped["corrected"] >= n_corrections]
    return pd.merge(filtered, CORPUS, on="original", suffixes=("_y", ""))[columns]

In [6]:
def get_by_metadata(corpus, **args): 
    """Filter the corpus based on the args provided"""
    if not len(args):
        return corpus
    query_str = " & ".join([f"{k} == '{v}'" for k, v in args.items() if k != "Lpoints"])
    print(query_str)
    filtered = METADATA
    if query_str:
        filtered = METADATA.query(query_str)
    if "Lpoints" in args and args["Lpoints"] != "all":
        query_str = f"lpoints >= {args['Lpoints'][0]} & lpoints <= {args['Lpoints'][1]}"
        print(query_str)
        filtered = filtered.query(query_str)
    user_lst = filtered['user_id'].to_list()
    return corpus.query("user_id in @user_lst")

In [7]:
def get_dict_corpus(corpus, size=10):
    """Return a part of the corpus based on the size"""
    return corpus[:size].T.to_dict("list")

In [8]:
def extract_list(df, str_sentence):
    return df[str_sentence].tolist()

In [9]:
def extract_pair_set(df):
    original_list = extract_list(df, 'original')
    corrected_list = extract_list(df, 'corrected')
    sentence_pair_set = {(original_list[i], corrected_list[i]) for i in range(len(original_list))}
    return sentence_pair_set

In [104]:
def annotation_search(
    final_corpus, agreementType, wordRange, nCorrections, sentenceType
):
    """
    This function takes as user input and returns the set of pairs (original sentence, corrected sentence).
    --------------------------
    Argument: final_corpus (final corpus in pandas data frame form)
              agreementType (Number of annotators who agreed with the correction - integer between 0 and 3)
              wordRange (Number of words in a sentence)
              nCorrections (number of corrections offered in the website)
              sentenceType (original / correct / both)
    Returns: a set of pairs (original sentence, corrected sentence) which meet the requirements specified
    """
    take_intersection = []  # contains the set of sentence pairs to be intersected
    if (
        agreementType > 3
        or agreementType < -1
        or (nCorrections != -1 and nCorrections < 1)
        or sentenceType > 1
        or sentenceType < -1
        or type(sentenceType) != int
    ):
        return set()  # return an empty set for invalid requirements
    elif agreementType >= 0 and agreementType <= 3:
        df = final_corpus[final_corpus["total_agree"] == agreementType][
            ["original", "corrected"]
        ]
        take_intersection.append(extract_pair_set(df))
    word_min = 0
    word_max = 100000
    str_list = []
    if wordRange != -1:
        word_min, word_max = wordRange
    if sentenceType == 0 or sentenceType == -1:
        str_list.append("original")
    elif sentenceType == 1:
        str_list.append("corrected")
    if sentenceType == -1:
        str_list.append("corrected")
    for str_sentenceType in str_list:
        df_min = pd.DataFrame(
            final_corpus[
                final_corpus[str_sentenceType].apply(lambda x: len(x.split()))
                >= word_min
            ][["original", "corrected"]]
        )
        df_max = pd.DataFrame(
            final_corpus[
                final_corpus[str_sentenceType].apply(lambda x: len(x.split()))
                <= word_max
            ][["original", "corrected"]]
        )
        take_intersection.append(extract_pair_set(df_min))
        take_intersection.append(extract_pair_set(df_max))
    if nCorrections > 0:
        df = final_corpus.groupby("original").nunique()
        original = set(df[df["corrected"] == nCorrections].index.tolist())
        original_df = pd.DataFrame(original, columns=["original"])
        original_count = pd.merge(
            original_df, final_corpus, how="left", on=["original"]
        )
        take_intersection.append(extract_pair_set(original_count))
    if len(take_intersection) == 0:
        return extract_pair_set(final_corpus)
    else:
        ss = take_intersection[0]
        for i in range(1, len(take_intersection)):
            ss = ss & take_intersection[i]
    return ss  # a set of pairs (original, corrected)

In [114]:
a = [["Returning home, I looked up the reason on my phone but there seems to be various views on it, so I'd like to leave out the reason here in order not to confuse people by asserting erroneous reasons.","Returning home, I looked up the reason on my phone but there seem to be various views on it, so I'd like to leave out the reason here in order not to confuse people by asserting erroneous reasons."],["A couple of friends gave me some advices about check In at the Airport.","A couple of friends gave me some advice about checking in at the Airport."],["There are two people in Manzai.","There are two people in Manzai.A more natural sentence is: \"In manzai there are two actors.\""],["I got it!!","I made it!!"],["However, there is a bad point.","However, there is a negative point."],["Can you tell me the most impressive novel/poem and the reason?","What is the novel or poem that impresses you the most, and why?"],["I have to walk 15 minutes to get to the nearest station, and then I get on a train that takes me to Tokyo.","I have to walk 15 minutes to get to the nearest station, and then I get on  the train that takes me to Tokyo."],["I admire writing little stories in Russian.","I like writing little stories in Russian."],["If only I could have studied English since child, It would have been more easy to study in other country.","If only I could have studied English since I was a child. It would have been easier to study in another country."],["Someone is positive, someone is negative and someone is false, or it's no longer real!","Some are positive, some are negative and some are just plain false, or  at least no longer true !"],["I see meny Chinese traveler.","I see many  Chinese traveler."],["It's another question of posibility of this knights in real life like l in books with sugary sonets.","It's another question if possibility of these knights in real life are like the ones in books with sugary sonnets."],["My name is Elza.15 years old.","My name is Elza, and I am 15 years old."],["I study English with Drink-bar for about two hours.","I study English at the drink counter for about two hours."],["Anyways I am very sad now.","Anyways, I am very sad now."],["But I like un usual movie like Harry potter.","But I like unusual movies like \"Harry Potter,\" so I liked this one."],["And people who want to read the same content in English,","And for people who want to read the same content in English,"],["Though the result of the score was zero to six and our team loss, I enjoyed playing the game.","Though the score was zero to six and our team loss, I enjoyed playing the game."],["i enjoyed seeing hydrangeas in the rain.","i enjoyed seeing the hydrangeas in the rain."],["I love to do having duo hua(tofu pudding) .","I loved  having duo hua (tofu pudding) ."],["I will talk about this thing with the student who gave me the invitation.","I will talk about this with the student who gave me the invitation."],["My answer more like reading comprehension, just posted the relevant part from the raw material, but no more further analysis.","My answer was more like reading comprehension, I just posted the relevant part from the raw material, but there was no more further analysis."],["We could find a lot of clothes at  the rag shop.","We were able to find a lot of nice clothes at a secondhand clothing shop."],["One day, she bought boiled corn at a traditional market.","Recently, she had bought some boiled corn at a traditional market."],["I can accept if native speakers says \"your Eng is too terrible\".","I can accept if native speakers says \"your English is terrible\"."],["But tape come off by itself.","But then the tape came off."],["People are living longer now.Discuss the causes of this phenomean?","People are living longer now. Discuss the causes of this phenomenon?"],["The first reason is communicating skill.","The first reason is  communication skills."],["I stepped back enough from the place calculating the possibility of distance.","I stepped back far enough from the place calculating the possible distance."],["I've not experienced water was cut off more than two days for a long time.","I've not experienced a water was cut off that lasted for more than two days for  some  time."],["Basically, mango is  not growing in colder area so, we have to maintain higher temperature in greenhouse.","Basically, mangos do not grow in colder areas so we have to maintain a higher temperature in the greenhouses."],["The conjugations, grammars are complicated.","The conjugations and grammar are complicated."],["I had no idea about it and I'm wondering if it is strange in other countries or not.","I had no idea  people felt that way, and I'm wondering if it is strange in other countries or not."],["They ate flying fishs and some times, rainbows sky in ketchup sauce...","They ate flying fish and sometimes, rainbows sky in ketchup sauce..."],["She grabbed a little: \"Do you know what it is?\"","She grabbed a little: \"Do you know what it is?\""],["It is rainy day in LA.","It is a rainy day in LA."],["However,i realize that i have no time to come home.","However, I realized that I don't have the time to come home."],["Hey , everybody , I am very glay to meet you in lang-8 ; I think this platform is very useful to study foreign language for everyone of us !But , in this platform , we just can be write something , can't talk to everyone , but my friend Dani , she say ,  I should talk to people in English and ask lots of questions.","Hey everybody, I am very glad to meet you in lang-8. I think this platform is very useful to study foreign language for all of us! In this platform we can only write posts, and cannot talk to everyone. My friend Dani said I should talk to people in English, and ask lots of questions."],["I remembered little word and grammar","I remembered only few words and grammar"],["How would you phrase this in English?","How would you describe this in English?"],["That's why I can speak in Russian.","That's why I can speak  Russian."],["I thought the shop was loved from many people.","I thought the shop was loved by many people."],["exercises \"Crisis\"","\"Exercises Crisis\""],["It was there, I could call if I need it.","It was there, so I could make a call if I needed to."],["Dependability analysis is a science of studying of fault and failure in system.","Dependability analysis is the science of studying  fault and failure in a system."],["In the learning language process you can try to make friends but sometimes you won't be able to do it, because everyone is not gonna like you, also it's typical you fell bad  or embarrassed because of your mistakes, but you should to remember without mistakes then how could have been possible you learnt your own language?","In the learning language process you can try to make friends but sometimes you won't be able to do it, because everyone is not gonna like you, also it's typical you  feel bad or embarrassed because of your mistakes, but you should to remember without mistakes then how could it have been possible that you learnt your own language?"],["I don't know what is different between Prose and Novel, but maybe prose has more literary meaning.","I don't know what is different between prose and a novel, but maybe prose has more literary meaning."],["Some European countries admit the euthanasia.","Some European countries admit  to using euthanasia."],["\"Since\" and \"as\" explain reasons that the listener already knows, while \"because\" and \"for\" explain reasons that the listener doesn't know.","COMMENT: The Original is fine and it is unclear how the \"Correction\" above differs from the Original."],["After it Andy got to know better Miranda and finally Andy quit because the job didn't go with her personality.","After a while Andy got to know Miranda better and finally quit because the job didn't go with her personality."],["although my writing isn't so good, i will take part in the essay writing activity .","Although my writing isn't so good, I will take part in the essay writing activity."],["I am not saying the fancy route they are willing to walk on is something bad.","I am not saying the fancy route they are willing to walk  is something bad."],["I started with running in the tredmill for 20minutes, and do other excercise for muscle.","I started with running in the treadmill for 20 minutes, and do other exercise for muscle."],["It is already likely to speak some Japanese.","It is already likely to speak some Japanese."],["Pre1 is as hard as 80 at TOEFL PBT and 550 at TOEFL IBT.","SUGGESTION: This claim needs supporting evidence. Here is one way to cite evidence in support of the claim -  According to AUTHOR (YEAR) the Pre1 is considered roughly equivalent to a TOEFL PBT score of 80 and or a TOEFL IBTscore of 550."],["Now you've been warned, this article will contains nothing useful for your life.","Now you've been warned, this article will contain nothing useful for your life."],["The registration number led to an individual that the police knew very well.","The registration number led to an individual who the police knew very well."],["Thanks for your reading/correcting ","Thanks for your reading/correcting"],["I have to memorize many things and I have to stand almost all of my working hours.","I have to memorize a lot of things, and I have to stand almost all of my working hours."],["Actually, I had our first baby last year!!","Actually, we had our first baby last year!!"],["Last night, you safely back to Japan, right?","Last night, you returned safely back to Japan, right?"],["This particle indicates a concluded actions, that is not the past as we occidental intend it.","This particle indicates  concluded actions, that is not the past as we occidental intend it."],["Please imagine you driving alone on the road without streetlights in the dark countryside.","Please imagine  yourself driving alone on the road without streetlights in the dark countryside."],["Break first for this morning.","Breakfast  this morning."],["You might be able to have a nice result, when you have a good preparation for that.","You might be able to have a nice result, when you have prepared well."],["It means that youth has an excellent opportunity to improve its knowledge without boring lessons or wasting time for getting to classrooms.","That means that young people now have an excellent opportunity to improve their knowledge without boring lessons or time wasted commuting to school."],["The area above is a conscious area, the area below the underwater area is an unconscious area, and the iceberg represents the human \"heart\" itself.","The area above represents the conscious area, the area underwater represents the subconscious, and the iceberg represents the human \"heart\" itself."],["I also made some important decisions for my future.","I also made some important decisions for/about my future."],["He concludes that there is no appropriate tune that might be used as a national anthem, hence why he suggests Britain invented a new one.","He concludes that there is no appropriate tune that might be used as a national anthem, hence why he suggests Britain invents a new one."],["I bought a new notebook today and I very happy .","I bought a new notebook today and I'm very happy about it."],["But I have continiously woken up by aftershocks.","But I have been continuously woken up by aftershocks."],["Vesuvius mountain tram.","Vesuvius cable car."],["I used to make snowman and kamakura when I was childhood.","I used to make snowmen and kamakura when I was in my childhood."],["We carefully moved out from the tent, and tried to figure out what it was.","We carefully moved out, and tried to figure out what it was."],["If I'm the teacher, maybe I will forgive.","If I teacher, maybe I will forgive.If I were/was the teacher, maybe I would forgive ( who? the students? the person who is lying to you?)"],["I can hear your intermittent breathing.","I can hear your  breathing."],["I felt petty about it.","I felt  pity  for it."],["Now, I'm staying at my sister's house and cooking lunch boxes and dinner everyday for us two.","Now, I'm staying at my sister's house and cooking lunch boxes and dinner everyday for us two. (You could say: \"I'm staying (living) at my sisters home and prepare lunch boxes and dinner for the two of us every day\")"],["During my search I came across the multi-cooker.","During my search, I came across the multi-cooker."],["I knew the Japanese Yen is fallen down than before due to it, so I needed to book all the trip's booking as soon as I can before the Yen will be cheap more.","I knew the Japanese Yen was on a free-fall before it even happened, so I needed to book all the trip as soon as I can before the Yen suffers any more damage."],["Policemen didn't tell the parents about detail about the incident until solved it.","Policemen didn't tell the parents about the incident in detail until it's solved."],["I didn't use to live in the city .","I didn't use to live in the city ."],["Every time that an arabic country had a secular goverment , trying to keep away its citizens from radicalism and fanatic religious...","Every time that an Arab country had a secular government , trying to keep  its citizens away from radicalism and religious fanaticism..."],["Mamoru, who is \"modern Beethoven\", has decieved us for many years.","Mamoru, who has been called the \"modern Beethoven,\" has decieved us for many years."],["I like player who treat ball like her.","I like players who can treat the ball like her."],["Now I remember everything...","I can still remember everything (even now)."],["'Giza' means 'gizagiza' in Japanese, so it means radiate or jagged.","'Giza' means is short for 'gizagiza' in Japanese, so it means radiating or jagged."],["I was afraid of many things and decided to stay a backpackers' hostel where has many Japanese customers.","I was afraid of many things and decided to stay at a backpackers' hostel with many Japanese lodgers."],["I wish to make it even a little growth in this site.","I do wish that I can improve my spoken English from this website."],["I have visitors today afternoon and my floor is a mess..","I have visitors  coming this afternoon and my floor is a mess.."],["I rapidly responded: \"yes\" and then she bluntly prompted the teacher: \"are you going to finish the question?\".","I rapidly responded, \"Yes.\" Then, she bluntly prompted the teacher, \"Are you going to finish the question?\""],["My reason for thinking so is that there are multiple records in history where children, young adults, and adults die from being infected with Varicella.","My rationale is that there are multiple records throughout history where children, young adults, and adults have died from being infected with Varicella."],["So I need to get well before monday.","So I need to get well before Monday."],["Within ten years,I hope to be working in a import laboratory, and to be recognized worldwide.","Within ten years, I hope to be working in an import(ant?) laboratory, and to be recognized worldwide."],["I went to a hospital.","I went to a hospital. << this sounds like you randomly went to a hospital, as if you were taking a walk :) I went to the hospital. <<this sounds natural, as if you went to the hospital with a purpose."],["Click on the 'Like' button attached with a entry which you like at the left-down corner.","There is a 'Like' button displayed in the lower left corner of every entry. Click on it."],["Although the borders are well defined in the map, we wouldn't know when we would be leaving the Galician part or when we would be entering the Catalan or Spanish.","Although the borders are well defined on maps, we wouldn't know when we would be leaving the Galician part or entering the Catalan or Spanish."],["only know few words and could say ","only know few words and could say (The only few words that I knew were"],["I feel a little hassle, because My boss attend this party, but I'm so excited!","I feel a little hassled, because my boss is attending this party, but I'm so excited!"],["To be accurate she stepped on my piano and kept mewing.","To be accurate, she stepped on my piano and kept mewing."],["She was a failure dancer in Paris when she was young...","She was a failure dancer in Paris when she was young... it should be: \"she was a failed dancer\", or: she was a failure as a dancer"],["As a result of practicing meditation and good deed, he was able to become a master in the employing of expedient means.","As a result of practicing meditation and doing good deeds, he was able to become a master in [the employment of / employing] expedient means."],["Now,we didnt call each other ,since we dont know what to talk ...","Now,we don't call each other ,since we don't know what to talk about..."],["Chinese culture is incorporated into Chinese medicine theories, so many theories are not based on experimental evidence, thus hardly acceptable by most western scholars.","In an academic style - Chinese cultural beliefs have influenced Chinese medical theories, so many theories are not based on experimental evidence. As a result, they hardly acceptable to most western scholars."],["I wached the video was made by two English guys, father and his son.","I watched a video that was made by two English guys, a father and his son."],["Are there any suggestions?","Do you have any suggestions?"],["So my husband, my parents went see her act.","So my husband and my parents went to see it."],["Remain the importance of quality during service process.","There remains to be considered the importance of quality during the service process."],["After she went back home.","After she got back home."],["And each of them has folder with their name, agreements and CV.","Everyone has a folder with their name, agreements, and CV."],["It's an enough serious reaction for a comedy like that, I think.","I think there is  not such a serious reaction if there is comedy like that."],["Today I have work,it is too boring,today is Tuesday ,I have to work three day more in this week,,my colleague let me sing song in a year end party,","Today I have work, it is too boring. Today is Tuesday and I have to work three day more days this week..........My colleague let me sing a song at a New Years party."],["What do you have a traditional way for coping a cold in your country?","What is your traditional way of coping with the common cold in your country?"],["Luckily, in my city we have plenty of good coaches, and some of them works only with young children.","Luckily, in my city we have plenty of good coaches, and some of them work only with young children."],["For you, what means your life in the dorms?"," hat does  your life in the dorms mean for you?"],["Firstly, before she came into the examination room, she passed security check.","Firstly, before she came into the examination room, she passed the security check."],["It is talk about cancer desease.","It talks about cancer disease./It talks about cancer."],["This is the movie that recording the performance in West End, London.","This  movie  is the recording of the performance in West End, London."],["A phycology counselor","A Psychology Counselor"],["On the contrary, people in the past could simply be killed by a little fever.","On the contrary, people in the past have been killed by a simple little fever."],["We hanged out in the Kuala Lumpur city and went Bazaar to buy some food.","We hung out in  Kuala Lumpur  and went to the Bazaar to buy some food."],["Please tell me what do you think.","Please tell me what you think about it."],["Why I noticed the virus?","Why have I noticed the virus?"],["In fact, as a young kid she suffered from asthma and was no a good idea to spend too much time in a dusty office.","In fact, as a young kid she suffered from asthma and it was not a good idea to spend too much time in a dusty office."],["Hi Vlad!","Hi, Vlad!"],["This made children's hopitals and private clinics are overload.","This means  children's hospitals and private clinics are overloaded."],["Thus, if I have an opportunity to take someone to do what we want without caring about the budget, I'm sure that one is definitely my mom.","(I could not understand the meaning of this sentence.)"],["Many football supporters discuss on the topic of who the best footballer of all time was.","COMMENT: The Correction Above is fine. Here is one more choice - Many football fans like to talk about who the best footballer of all time was"],["School system in Russia is very simple.","The school system in Russia is very simple."],["So they sent their children to America or England for better education resources.","So they send their children to America or England for better education resources."],["We lock ourselvs in the room and did not see the world around us.","We lock ourselves in  rooms and do not see the world around us."],["in your being.","in your being."],["As everyone knows, there is a statue of Virgin Mary that sheds tears.","As everyone knows, there is a statue of the Virgin Mary that sheds tears."],["It is obviously that The Hoang Sa and Truong Sa ( the Paracel) Islands belong to","It is obvious that The Hoang Sa and Truong Sa ( the Paracel) Islands belong to"],["My hobbies are traveling,driving,shopping,watching baseball game!","My hobbies are travelling, driving, shopping and watching a baseball match!"],["Finally, they've sung a second opening song of My Hero Academia season2.","Finally, they sang the second opening song of My Hero Academia season 2."],["It said:","It said:"],["It took me the whole morning today to fix the stupid mistake yesterday.","It took me the whole morning today to fix the stupid mistake yesterday."],["However, instead of being overwhelmed by bad moods, I would like to share with you two","However, instead of being overwhelmed by a bad mood, I would like to share with you two"],["Now that I grew up, I can see what's going on our country.","Now that I have grown up, I can see what's going on in our country."],["It is very hard to me to understand what provision says.","It is very hard  for me to understand what provision says."],["I mean, there is movies with good plots and bad plots, but only with good plots survive at long time at cinema.","I mean, there  are movies with good plots and bad plots, but only those with good plots survive  a long time  in the cinema."],["It's a traditional custom of the day.","It's a tradition."],["Next morning, we went to Ho Chi Minh City where has many European and American style structures.","Next morning, we went to Ho Chi Minh City which has many European and American style structures."],["If you get the email from ex-boy/girl friend whom broke up with you about 10years ago.","If you get an  email from an ex-boy/girl friend who broke up with you about 10 years ago."],["That is why academic harassment is a problem which is difficult to solve.","That is why academic harassment is a problem that is difficult to solve."],["The Kinh is the most populous ethnic in Vietnam.","The Kinh  are the most populous ethnic group in Vietnam."],["When Japanese says simply \"sauce\", it means worcestershire sauce.","When Japanese says simply \"sauce\", it means Worcestershire sauce."],["The Australian governement's chief has made these optimist statements after that several objects was found in the research area.","The Australian government's chief has made these optimistic statements after  several objects were found in the research area."],["This situation is you have already ordered the pizza, but it do not serve yet.","This situation is when you have already ordered the pizza, but it has not been served yet."],["So, while I was looking for books, I was so afraid that my illegal parking was found by police, but, in last,  police did not come.","So, while I was looking for books, I was afraid that my illegal parking would be found out by the police, but in the end, the police did not come."],["i am planing to learn car driving.","I am planning on learning to drive."],["So, I'm trying to do something incredible stuff on this vacation.","So, I'm trying to do something incredible stuff on this vacation. (So, I will try to do something incredible this coming vacation.)"],["How stupid I am to do such a thing!","How stupid for me to do such a thing!"],["I look forward to talking with you then.","I look forward to  speaking with you then."],["I minored in chemistry and also attended classes in faculty of technology and science.","I minored in chemistry and also attended classes in the faculty of technology and science."],["Children would be too reliant on the rules when they make decisions to have little ability handing problems independently.","Children would be too reliant on the rules when they make decisions; their ability to independently handle problems would be very poor."],["Oni(Ogres) came to the town and to do bad things.","Oni(Ogres) came to the town  to do bad things."],["so I'm meeting my friends from university this evening.","So I'm meeting my friend from university this evening."],["I now worry about whether the above ticket will be fully sold out until I finish to update my passport.","  I'm worried that the above ticket will be fully sold out  before  I'm finished updating my passport."],["She's popular in the class, never shows her weakness and keep the disease secret to everyone.","She's popular in  class, never shows her weakness, and keeps the disease a secret to everyone."],["It has hundreds years age.","It is hundreds of years old."],["It was unexpectedly, but cool.","It was unexpected, but cool."],["Spending a few months or even a year to seeking a job makes me happy is much better than spending a lifetime trapped in a place that we are reluctant to do because it's just as safe and stable.","Spending a few months or even a year to seek a job that makes me happy is much better than spending a lifetime trapped in a job that we are reluctant to do (just) because it's  safe and stable."],["I should have pondered  my future in 20 years old.","I should have pondered my future at 20 years old."],["Korea), so I speak a standard Korean.","Korea), so I speak  standard Korean."],["If that's the case the traffic stops also.","If it does, transportation will stop again."],["I already started to break paradigms in my mind and it has been wonderful...","I have already started to change the paradigms in my mind and it has been wonderful..."],["And I didn't think about what I wanted to do in university until I entered high school.","I didn't think about what I wanted to do in university until I entered high school"],["They are having extorted the common people.","They  have extorted the common people."],["Second, various restaurants are increasing with a rapid speed.","Second, various restaurants are increasing with a rapid speed."],["Maybe someone is making a historical discovery somewhere on the earth.","Maybe someone is making a historical discovery somewhere on the earth right now."],["The deeper we go through and feel something, the better our brains are.","The deeper we go through and feel something, the better our brains are."],["It must be very short term..","It must be very  rarely.."],["Today, from noon time, Ibegan to swap  summer and winter clothes.","Today at noon , I began to swap my summer and winter clothes."],["Finally, drugs can relieve sufferings and stress.","Finally, drugs can relieve suffering from stress."],["At first I thought it is French, but it was English.","At first I thought it  was French, but it was English."],["I love this festival,because in this time we have three holidays.","I love this festival because we have three holidays around this time."],["I watched a video on youtube.","I watched a video on YouTube that talked about how to learn English."],["Final dog's name was Pino.","The last one's name was Pino."],["After the accident (as you know), I began to have to stories in my life.","After the accident (as you know), I began two careers ."],["There are 22 member in my laboratory  and I felt that a lot of member are in this lab.","There are 22 members in my laboratory and I felt that a lot of members are in this lab."],["I am worried about the death of Lang-8 to come.","LOGIC: Since Lang-8 is not a biological organism, this wording is better - I am worried about the potential demise of Lang-8 ."],["My explains are expression in Japanese after all.","My explainations are all expressed in Japanese after all."],["Nowdays A lot of young people want to be a celebrity.","Nowadays, a lot of young people want to be celebrities"],["Why people always hurt each other?","Why are people always hurting each other?"],["I knew that people communicate each other in English as a matter of course but I couldn't feel it.","I knew that people communicate to each other in English as a matter of course, but I couldn't feel it."],["I am just listening without singing.","now I am just listening without singing."],[" Which sentence is suitable? --- \"be going to\" and \"will\"","Which sentence is better? --- \"be going to\" or \"will\""],["Equally importantly, the enlargement of fast food chains is a root of the increase in average weight of people.","Equally importantly, the enlargement of fast food chains is a root of the increase in the average weight of people."],["Today, I'll talk about the thing which I think about learning languages.","Today, I'll talk about what I think about learning languages."],["In fact ,these are different about talking with peaple and leaning foe examination .","In fact, talking with people and leaning for examination are two different things."],["It is one of the famous concepts of Japanese culture.","It is a famous aspect of the Japanese culture."],["Howevwr,I'm lucky that my family are living.","I'm lucky that my family is still living."],["I feel a little boring now.","I feel a little bored now."],["We apologise for inconveniences this may cause.","We apologize for any inconvenience(s) this may cause."],["\"Put these beefs and vegetables into the pot when it becomes boiling.","\"Put the beef and vegetables into the pot and when it boils"],["And I still have some holiday home work didn't finish Y-Y.","I still have some holiday homework I didn't finish Y-Y."],["Can you remember?","Did you remember?"],["I go Himeji by bicycle almost every week, so it is not special.","I go to Himeji by bicycle almost every week, so it is not special."],["I read it in Japanese,though...","I read it in Japanese, though."],["Why China called Mandarine ???","Why China called Mandarin ???"],["The all attendants have to be careful even on the way to and from the wedding party other than during it.","The all attendants have to be careful even on the way to and from the wedding party other than during it."],["There is also a large hall and private rooms.","There  are also private rooms and  a large hall ."],["I believe everybody should have good wages in order to be able to live normal live.","I believe everybody should have good wages in order to be able to live normal live."],["I will took my radio wrist watch and confirm whether is it possible to repair or not.","I will take my radio wrist watch to confirm whether is it possible to repair or not."],["I didn't expect that you would like to listento the songs by Wangfei.","I didn't expect that you would like to listen to songs by Wangfei."],["As I wrote in my diary of the other day, water problem is serious.","As I write in my diary about  the other day, I want to point out that the water problem is serious."],["you are my loophole against all this bullshit !","you are my escape from all this bullshit !"],["Every time I visited my grandparents' house during a summer vacation once a year, I was so excited for all only having thought  that I was able to take a train for a long time.","Every time I visited my grandparents' house during summer vacation once a year, I was so excited that I was able to take a train for once in a long time."],["Many people say it is important that playing sports in youth.","Many people say it is important that we play sports in our youth."],["If it's really interesting job, I would accept it.","If it's a really interesting job, I would accept it."],["The earthquake often happens in parts of Japan.","Earthquakes often happen in parts of Japan."],["Scondly, Preventing Flooding through the Rainwater Drainage.","Secondly, the prevention of flooding through the rainwater drainage system."],["That rally makes me interested.","That rally makes me interested."],["Because usually I need to wait for holiday to go back to see my parents.","Because usually I need to wait  until holidays to go back to see my parents."],["I also think that she is a business lady.","I also think that she is a business lady."],["Especially, This project for developing countries and least developed countries people."," This project was especially for people in developing countries and least developed countries "],["In practice I don't know her name.","In practice I don't know her name."],["The seasonal tradition called \"fubustushi\" in Japanese.","The seasonal tradition called \"fubustushi\" in Japanese."],["And therefore, they claim that there is insufficient evidence to deduce the suggestion of vitamin D being beneficial to the reduction of respiratory infections in individuals.","And therefore, they claim that there is insufficient evidence to deduce the suggestion of vitamin D being beneficial for the reduction of respiratory infections in individuals."],["\"He is always crazy that he think to go out with the girl.\"","\"He is always crazy thinking to go out with the girl.\""],["If I have not a headache, I would like to go out with my new bicycle!","If I hadn't had a headache, I would have liked to go out on my new bicycle!"],["\"Marushin Hanten\" is recommended by Takuzo Kadono, an actor who publishes gourmet books about Kyoto.","Marushin Hanten is recommended by Takuzo Kadono, an actor who publishes gourmet books about Kyoto."],["I use English now.","I use English now."],["I found someone is leaving free pallets in some garbage place for free to take.","I found someone who is leaving pallets in some garbage place; they were free to take."],["Candidates for reading novels are Monsieur Poirot's File and Mr.","Candidates for  novels are Monsieur Poirot's File and Mr."],["Come on, you little devil.\" ","Come on, you little devil!\""],["Hiroshima has some world heritages and some view spots.","Hiroshima has some world heritages and some good view points."],["For what it's worth, here's a link of the advanced search function.","For what it's worth, here's a link to the advanced search function."],["Installing a program on your computer is just required to produce bitcoins.","All that is required is installing a program on your computer."],["I must get over this things if I want to acess a success!","I must get over this if I want to succeed!"],["I don't offense him so much.","I don't bother him very much."],["We exchange books and some news from our life, but that is all.","We exchange books and news about our lives, but that is all."],["if you are learning Chinese,we could make friend help each other.","If you are learning Chinese, we could become friends and help each other."],["I finished reading an interesting book called \"The Stanford Method for Ultimate Supre Recovery\".","I finished reading an interesting book called \"The Stanford Method for Ultimate Super Recovery\"."],["That place is not what I want to go.","That place is not where I wanted to go."],["But she still has a hope.","But she still has  hope."],["I take a bath from now.","I will take a bath from now."],["Hello everyone, I wrote some questions  to interview foreign people in my country, but I don't know if it is right ","Hello everyone, I wrote some questions to interview foreign people/foreigners in my country, but I don't know if  they are right "],["You mustn't/cant feed animals in the zoopark.","You mustn't/can't feed animals in the zoo."],["Who does want to meet a sweaty and ugly man in a suit in a hot summer?","Who  wants to meet a sweaty and ugly man in a suit in  the hot summer?"],["I have watched movies with Japanese subtitles.","I have watched movies with Japanese subtitles."],["Rare but could be one reason for people to go to university is that they are advised or requested to do, however it is not their choice.","It is rare but  one reason  people to go to university could be that they are advised or requested to do so; however it is not their choice."],["Can we say : in many different cultural environment ?","Can we say : in many different cultural environments ?"],["I think I will be fat guy soon.","I think I will become a fat guy soon."],["In conclusion, I believe that lying for a good reason and not saying the truth it actually help a lot the relationship between people.","In conclusion, I believe that lying for a good reason and not saying the truth  actually helps a lot the relationships between people."],["When I have lived in America, sometimes I have got confused about Japanese-English words.","When I lived in America, sometimes I have gotten confused about Japanese-English words."],["Because I am busy, I use a Fude-Pen habitually exclusively recently.","Because I am busy, I have been using a Fude-Pen recently."],["I went to watch the movie \"Big Hero 6\" as soon as it been released in the China last weekend.","I went to watch the movie \"Big Hero 6\" as soon as it was released in China last weekend."],["At the end, she told us she was quite impressed because we've done good progress.","At the end, she told us she was quite impressed because we've made good progress."],["Also, our figures are the same, so we can share clothes.","Also, our figures are the same, so we can share our clothes."],["Hello,everyone,i want to improve my English by communicating with people whose native language is English.","Hello, everyone. I want to improve my English by communicating with people whose native language is English."],["It's a compulsory regulation in our school.","It's a compulsory regulation in our school.(Maybe you could say: It's a rule set by my school.)"],["I sometime do it.","I sometimes draw."],["The scientists in Hungary announced that there is also a possibility that the frequent use of smartphones have negative effects on sperms.","The scientists in Hungary announced that there is also a possibility that the frequent use of smartphones have negative effects on sperm."],["So, I have questions for you: Did you learn cursive writing at school and are you using it now?","So, I have a question for you: Did you learn joined up handwriting at school and are you using it now?"],["Today is the match between MU and PSV, i has no preferences to anyone but i just want to see a interesting game.","Today is the match between MU and PSV, i have no preference i just want to see a interesting game."],["Other thing has changed in my life is that I did exercise every day but now I do exercise once or twice a week.","Another thing that has changed in my life is that I exercised every day, but now I only exercise once or twice a week."],["I found the English I learned at Japanese school is incorrect.","I found that the English I learned at school is incorrect."],["However, \"Gardens By the Bay\" is something else and it has a surprisingly huge site.","However, \"Gardens by the Bay\" is something else and is surprisingly huge."],["I was so surprised and for a few seconds I could not react.","I was so surprised  I could not react."],["I'm going to write about favorite clothes again.","I'm going to write about my favorite clothes again."],["It is a fixed schedule.","(See below)"],["So I induced my husband to visit his uncle's house by even only two of us, taking advantage of preaches from the priest to support my intention.","So I got my husband to agree to visit his uncle even with just the two of us, taking advantage of the priest's sermon/preaching to support my intention."],["I want to speak English well though this website.","I want to speak English well through this website."],["We learn that lives are more important that profit and look inside us and finecomb out all our hidden prejudices and privileges.","We are learning that lives are more important that profit and to look inside ourselves and pick out all our hidden prejudices and privileges."],["If I feel comfortable, I properly study English again.","If I feel comfortable, so I'll properly study English again."],["just so many trees.","there are just so many trees."],["The only option if you want to go in that time is catching a intercity train which is way too expensive.","The only option you have if you want to go in that time is catching a intercity train which is way too expensive."],["So he decided to leave there his little cousin who was a great cook and who had pretty much his same age of the soothsayer; while, he would have kept travelling the world in search of a cure for his hand.","So he decided to leave  his little cousin there.  His cousin was a great cook and   was pretty much his same age  as the soothsayer.  The man would  keep traveling the world in search of a cure for his hand."],["And I heard that by watching tv eat food is not good for health.","Also, I heard that watching TV while eating is not good for your health."],["Today I want to the library borrow a Europe trip book.","Today I went to the library to borrow a European trip book."],["It became popular and be imitated by others.","It became popular and  imitated by others."],["\"Going to Oni-ga-shima.","\"I am going to the Oni-ga-shima."],["In the video, a technology expert who names Katie has had an experience of staying in the hotel which offers an unique service.","In the video, a technology expert named Katie has had an experience of staying in a hotel which offers a unique service."],["I have a very pharynx pain since this morning.","I have a very   sore throat since this morning."],["I went to practice the horn with mother's brass band.","I went to practice the horn with the mother's brass band."],["Virtually, my grade of mid term had been supposed to be more 90% in the class.","My grade for my mid term had been supposed to be more 90% in the class."],["Now I chose \"Micheal Jackson Man in the mirror\" now.","I chose Micheal Jackson's \"Man in the Mirror\" just now."],["I sometimes see a conversation sentences written in Japanese like this in this site.","I sometimes see conversational sentences written in Japanese like this on this site."],["The first priority is to choose yourself.","The first priority is to be there for yourself."],["Main character had fought as a soldier of US with his big brother until he was eaten by a Kaiju.","The main character had fought as a soldier for the US with his big brother until his big brother was eaten by a Kaiju."],["Today is the reservation day of eye doctor.","Today I had my appointment for the eye doctor."],["it's so excited!","I'm so excited!"],["I'm looking forward to come summer.","I'm looking forward to  summer. OR \"I'm looking forward to the upcoming summer.\""],["Actually it was the first time we went there, but all the souvenirs were cute.","Actually, it was the first time we went there / it was our first time there, and all the souvenirs were cute."],["They are also skilful to respect.","They are also skillful to respect."],["Love makes is happy and excited in the beginging, a shining dimond.","Love makes me happy and excited at first, like a shining diamond."],["I felt obliged to celebrate the success together with my coleagues.","I felt obliged to celebrate the success together with my colleagues."],["I am going to visit Kyoto next weekend to go the event named KYOTO EXPERIMENT : http://kyoto-ex.jp/","I am going to visit the KYOTO EXPERIMENT event in Kyoto next weekend. http://kyoto-ex.jp/"],["I'm interested in other country's culture, language,and history.","I'm interested in other countries' culture, language and history. - or - I'm interested in the culture, language and history of other countries."],["Today I took 'Eiken' 1st grade exam and it was kind of challenging.","Today I took the 'Eiken' 1st grade exam and it was kind of challenging."],["What happen my dog in this  morning.","What happened to my dog  this morning."],["I, nonetheless, contend there will be many benefits for the children if their parent spent their free time with their children for funny activities.","I, however, contend that there are many benefits for children whose parents spend free time with them doing fun activities."],["Fortunately I think that we are on time to change some behaviors or activities to build a better world.","I think that we are fortunate to have some time to change some behaviors  and activities, in order that we may build a better world."],["Also, you can recognize your transition of interest.","Also, you can recognise your patterns of interest."],["When I was in Saint Peterburg, I had a lot of free time.","When I was in Saint Peterburg, I had a lot of spare time."],["because TV always broadcast many different types of movies.","because TV always broadcasted many different types of movies."],["I am beaten, because I have been caught up in some trouble of my job recently.","I am beaten, because I have been caught up in some trouble at work recently."],["It can make people start to respect people in every field.","People will start showing more respect towards people in other professions."],["Good girls are bad girls that haven't been caught.","Good girls are bad girls that haven't been caught."],["I recommend Karuizawa to visit for two or three days.","I recommend Karuizawa  for a 2-3 days visit."],["But in the Orient, I think that I was recognizing the importance of unconscious in the course of pursuing \"enlightenment\" several thousand years ago.","However, in the Orient, I think that we recognised the importance of unconscious in the course of pursuing \"enlightenment\" several thousand years ago."],["In the kitchen there are bowls with food and drink for the dog and a full fridge, in addition there is constantly something baking in the oven.","In the kitchen there are bowls with food and drink for the dog and a full fridge. In addition there is constantly something baking in the oven."],["We will have a 8 days holiday from September 30th to October 1st,I'm very happy!","We will have an eight day holiday from September 30th to October 7th. I'm very happy!"],["It was quite difficult for me to understand his English.","It was quite difficult for me to understand his English, (connect to the following)"],["What I did today.(Dialy)","What I did today. (Diary)"],["Which kind of emotion does s/he has?","Which kind of emotions does s/he have?"],["I worried about the pollution of the sea because of the dirty water from the plant.","I was worried about the pollution of the sea from the dirty water leaking from the plant,"],["Then, we shout loudly \"Fortune in !","Then, we shout loudly \"Fortune in!"],["I use this website to practice on how to use English language.","I use this website to practice  how to use English language."],["In other words, moral obligations and duty required for Samurai have priority.","In other words, the moral obligations and duties required of a Samurai have different priorities."],["Have a nice dream, thank you for reading my journal.","Sweet dreams. Thank you for reading my journal."],["Overtime work has been longer the former job.","Overtime work has been longer than former job."],["His tearful cutoff animation became very popular, and he started the new career as a cutoff animation creator.","His tearful cutoff animation became very popular, and he started a new career as a cutoff animator."],["What did make your pronunciation better?","What  made your pronunciation better?"],["It will be the first time I live alone.","It will be the first time I will be living on my own."],["The same to us when we think something is unchangeable.","The same  is true for us when we think something is unchangeable."],["When I showed the  flower's photos to a person who is keen to hydrangea, he said it could be a kind of natural mutation and he had never seen such a flower.","When I showed the flower's photos to a person who likes hydrangea, he said it could be a kind of natural mutation and that he had never seen such a flower before."],["Did you like it?","Did you like it?Perfect!"],["I was surprised when I log in to Lang-8 because it has changed a little.","I was surprised when I logged in to Lang-8 because it has changed a little."],["But on thee ather hand.","But on the other hand."],["But today I did not can exercising.","But today I couldn't exercise."],["I don't know what to write.","I don't know what to write about."],["According to forecast on the economic development in the Euro Union, in 2016 the economy of the region is recovering at moderate pace.","According to a forecast on the economic development in the European Union, in 2016 the economy of the region is recovering at moderate pace."],["m a day.","m every day."],["My answer didn't worth a single glance to him.","My answer wasn't worth a single glance to him."],["Thus, I am very thankful the professor Ogburn giving me this chance.","Thus, I am very thankful to Professor Ogburn for giving me this chance/opportunity."],["This picture is a hot pot made by soy milk!","This picture is of a hot pot made with soy milk!"],["Historically, there has been a lot of foreign people, because it thrived as a port.","Historically, there  have been a lot of foreign people, because it thrived as a port."],["I hope find a job, because this trip is really important for me.","I hope to find a job, because this trip is really important to me."],["This song is slow and melody is much to me.","This song is slow and its melody is much to me."],["T:that is also firecracker sounds.","T: Those were also firecrackers."],["I met with my ten year old niece on the New Year's Day.","I met with my ten year old niece on New Year's Day."],["I always stand by myself.","I always stand by myself."],["And why?","Why?"],["in order to be like a native speaker.","in order to  speak like a native speaker."],["That are good partners from way back.","They can be good partners for a long time."],["Sunday, February 09 2013","Sunday, February 09, 2013"],["Question to you ","Question for you"],["When I graduate university.","When I graduated from university, I went overseas for the fist time."],["As you can see, we did almost nothing special in Baguio.","As you can see, we did almost nothing special in Baguio. (Or, we didn't really do anything special.)"],["I have to study all day to prepare for the exam.","I have to study all day to prepare for an exam."],["Actually our team lost at the first game of the local preliminary round, but it was really exciting.","In fact, our team lost at the first game of the local preliminary round, but it was really exciting."],["I remember when i was i child i had a close friend.","I remember when I was a child I had a close friend."],["According to this case I am a very happy as a common people, and I feel pity for them.","According to this case I am a very happy as a common people, and I feel pity for them."],["There were a lot of jelly fish water tanks and some of them were lighting and changing the color.","There were a lot of jellyfish water tanks, and some of them were lit up and changed color."],["There are list on the left side and your tasks depending on one of the list are on the right side.","There are lists on the left side and your tasks, which depend on (selected from?) one of the lists, are on the right side."],["There is a theory which I tought up!","There is a theory that I thought up!"],["I've studied English since childhood, and majored in Chinese language and literature at college.","I've studied English since childhood, and majored in Chinese language and literature at college."],["The other is three words\"R,U,N\".","The other three words are \"R,U,N\"."],["And living with a friend may be good for my life, there is one person talking to me, we can go to play basketball together at weekend.","And living with a friend may be good for  me. There is one person who can talking to me and we can go  play basketball together  on the weekend."],["But it's not just forest there where I live : the city center is located lower from our residential area, so we also have a nice view ( especially at night ).","But it's not just forest there where I live : the city center is located in a lower elevation from our residential area, so we also have a nice view ( especially at night )."],["Please welcome!)","Please feel welcome to contact me!"],["If parents want their children to have a better future, they should prompt their children's passion rather than force them to do something they might dislike of.","If parents want their children to have a better future, they should promote their children's passion rather than force them to do something they might dislike ."],["I am timid.","I am timid"],["My possessions are really few.","I have very few possessions."],["In conclusion , I think leisure time is important to improve the your maind and your talente .","In conclusion , I think leisure time is important to improve the your mind and your talent."],["It will be lovly sunday.","It will be lovely Sunday."],["The signs could be tricky, because on some crossroads the flour was basically an unnoticeable dot, or on the back of trees you could hardly spot.","The signs could be tricky, because on some crossroads the flour was basically an unnoticeable dot, or on the back of trees, so you could hardly spot it."],["I am determined to make good body shape.","I am determined to get a good body shape."],["Only in this way, Baymax can send them away and leave himself sleeping in the space machine with nothing worries.","Only in this way can Baymax send them away and leave himself sleeping in the space machine with no worries."],["I bet you know of sushi, and conveyor belt sushi is offered by restaurants where sushi is carried on the conveyor to take it when you want.","I'm sure you know of sushi, and at conveyor belt sushi restaurants, the sushi is carried on a conveyor belt, and you can take what you want."],["Oh, I'm not suppose to buy such many because I want to keep fit.","Oh, I'm not supposed to buy so many because I want to keep fit."],["Although it is cold outside, I don't feel the cold at all while walking.","Although it is cold outside, I don't feel cold at all while walking."],["Or perhaps I change really a little.","Or perhaps there is only a little difference."],["My family went out to go to school and work .","My family went out to  to school and work."],["Eventually, I had no choice but to bound for Japan.","Eventually, I had no choice but to head for Japan."],["Change is a good way to become a better woman.","Changing your habits is a good way to become a better woman."],["The most important thing in that chapter is the encounter between Mr Utterson and Mr Hyde.","The most important thing in this chapter is the encounter between Mr Utterson and Mr Hyde."],["Now,there are many international problem.","Now i take into note that there are many international problem."],["We usually go to the photo studio to have our picture taken once a year since my husband and I got married.","Since my husband and I got married we usually go to the photo studio once a year for a new portrait."],["It has been five weeks since I had compression fracture of spine.","It has been five weeks since I had (or got) spinal compression fractures."],["Then I read a book in a starbacks.","After leaving the garden, I then read a book in a Starbucks."],["From last weekends, it's been raining everyday.","Since last weekend, it's been raining every day."],["Genial and eccentric professor of zoology Vladimir Ipatievich Persikov accidentally finds amazing  phenomenon red light that acts to emryo.","The genial and eccentric professor of zoology Vladimir Ipatievich Persikov accidentally finds an amazing phenomenon - a red light that reacts to embryos."],["don't erase the other,","doesn't erase the other,"],["My english teacher said I had a little improvement in conversation.","My English teacher said I had a little improvement in conversation."],["As I got to my home, my body had become so cold like a dead one.","As I got to my home, my body was so cold like a dead body."],["But a better understandly came to us while they listened and done them increase by ok and harmony.","But a better understanding came to us while they listened and increased the peace and harmony."],["Everybody knows that being a good sales manager means to be Casanova.","Everybody knows that being a good sales manager means to be a Casanova."],["To verify it's authenticity we will analyse the pressure, shape and slant of the handwriting as we have accomplished to do so with several messages to your mistress.","To verify its authenticity we will analyse the pressure, shape and slant of the handwriting as we have managed to do  with several messages to your mistress."],["After walking for about 20 minutes, I finally saw a woman and asked her how to go.","After walking for about 20 minutes, I finally saw a woman and asked her how to get there."],["I felt out of it..","I felt out of it."],["So I cancelled all the itineraries for my trip to France.","So I cancelled all of the itineraries for my trip to France."],["Her daughter was 3 years senior than mine.","Her daughter is 3 years older than mine."],["I'd like to write about this theory more today.","I'd like to write more about this theory today."],["It came to my mind that it might be good to spend the second half of my life time in another country.","It came to my mind / I figured that it might be good to spend the second half of my life in another country."],["I thank that she has supported me who always busy with work.","I'm thankful that she has supported me who is always busy with work."],["my first diary!!","My first diary entry!"],["How many stock company in Japan?","How many stock companies are in Japan?"],["Especially,  exhibition of reptile was very exciting.","Especially, The reptile exhibition  was  especially exciting."],["My friend told me to join his team about cibersecurity.","My friend told me to join his team about cybersecurity."],["When I was listening to one of my favorite podcasts named \"An Encounter That Changes Your Fate\" (I translated the Japanese title into English) this morning, the person, who is a kind of a spiritual mentor, recommended walking at night.","When I was listening to one of my favorite podcasts named \"An Encounter That Changes Your Fate\" (I translated the Japanese title into English) this morning, the host, who is a kind of a spiritual mentor, recommended walking at night."],["It was passed 5 years from that day \"3.11\".","Five years have passed since that day, 3.11"],["They told me that almost every Korean family made this food to meet the new year, and it was a symbol of gaining one year old for Korean Teenagers.","They told me that almost every Korean family made this food to greet the new year, and it was a symbol of being one year older for Korean Teenagers."],["To strain to defecate it seems pained.","  It hurts when I try to defecate."],["They're dogs of certain countries.","Certain countries are bastards. "],["I called the airport lost property service and they gave me an email address and I sent.","I called the airport lost property service and they gave me an email address and I sent a message."],["Since the war is starting, many citizens flee their country to next country which is mostly Lebanon and Jordan.","Since the war started, many citizens fled from their country to a neighbour country which is mostly Lebanon and Jordan."],["Moreover, the most known newspaper company or information agency have their own website so that users can find the most recent news and follow their evolution.","Moreover, the most well-known newspaper companies or information agencies have their own websites so that users can find the most recent news and follow its evolution."],["At, first we were little nervous.","In the beginning, we were a little nervous."],["I have small lumps in my stomach,moxibustion is warming the lumps.","I have small lumps in my stomach, moxibustion is warming the lumps."],["SALE is always wonderful!","A SALE is always wonderful!"],["The objective is for my practice."," for my practice."],["that is \"Indian Camp\" written by Hemingway."," The book/story is \"Indian Camp\", written by Hemingway."],["Don't belive all these bobo who want to make you believe the opposite .","Don't all  bobos who want to make you believe the opposite ."],["He may be a head of a department.","He may/could be a/the head of a department."],["This is an obscure concept in Chinese means \"elf\".","This is an obscure concept in Chinese, meaning \"elf\"."],["Also, patients must permit them and this low is a big obstacle from that.","Also, patients must permit them, and this law is a big obstacle to that."],["As a matter of fact, this phrase comes from some of the old Japanese words which were used in middle part of Japan about 1000 years ago.","As a matter of fact, this phrase comes from some of the oldest Japanese words which were used in middle part of Japan about 1000 years ago."],["Most students use mobile phones for learning knowledge, accounting for 59.5%.","LOGIC: What people report they do is often different from what they actually do. \"Social desirability bias\" makes self-reported claims uncertain. 59.5% of the students in this sample reported using mobile phones to \"gain knowledge.\""],["First, homework is good to be used for student's review.","Homework is a good tool for students to use to review for a test."],["The movie was made for about 30 years ago.","The movie was made  about 30 years ago."],["I could speak English at that time.","I could speak English correctly at that time."],["One day other people said we  got resemble each other ,","One day other people said we got resemble with each other ,"],["In my child days, I'd sometimes thought about that and been in fear."," During my childhood,  I sometimes thought about that and  was fearful."],["Now, I would like study english...","Now, I would like to study English..."],["I reading a 'Outliers' written by Malcolm Gladwell.","I am reading  'Outliers' written by Malcolm Gladwell."],["Otherwise, nothing is not different from something.","Otherwise, existence is no different from nothingness."],["We spend many time together.","We spend  much time together."],["At first we went to Asakusa temple then we went to drinking.","At first we went to Asakusa temple then we went  drinking."],["Fortunately I had a wonderful brother who has been to school.","Fortunately, I had a wonderful brother who had been to school."],["It makes the school lively.","It gives the school a lively atmosphere."],["Not only said to me that i have to do homework, she knows how to motivate me to wish to study and encourage with praise words when i had done well.","Not only did she say  to me that I had to do homework, she knew how to motivate me to study and encouraged me with praise when I had done well."],["Today, I'm gonna talk about my trip to the USA and the experiences.","Today, I'm gonna talk about my trip to the USA and my experiences."],["I want to get along well from now on.","I want us to be friends forever."],["i can't stand it any more. ","I can't stand it any more."],["I think that not only me  being puzzled by it,what about you?","I think I am not the only one puzzled with it ,what about you?"],["When I cleared out in my room, I fond missed item.","When I cleared out  my room, I found an item that had been missing."],["I will work in post office as a cleark at the window.","I was hired to work as a post office clerk, serving customers."],["However, the one was already closed.","But it was already closed."],["You've desparately effective been to save Mayuri.","Your desperate, barely ineffective bits to save Mayuri."],["Writting essay","Written essay"],["when I read diary which another person write, I find \"xoxo\" ","when I read a diary that another person wrote, I found \"xoxo\""],["Last work, some shipper import a lot of things from Overseas.","Lastly, I help some shippers to import a variety of items from overseas."],["I think maybe it is atracted your heart for something of big.","I think maybe it is atracted your heart for something of big."],["When I see such stories, they make me very happy because I like to see the technology being used to help people in need.","When I see such stories, they make me very happy because I like to see technology being used to help people in need."],["maybe i think that so cold in my office than outside.","I think that it was warmer in my office than outside."],["So some of my friends sometimes goes there just having Sento.","So sometimes my friends go just have a bath."],["A Japanese pop star shaved her hair over sex scandal.","A Japanese pop star shaved her hair  because of a sex scandal."],["I asked him \"Can't making sake is systemized?\" because my research is connected with systemization in industry.","I asked him \"Can't sake-making be systematized?\" because my research is connected with systematization in industry."],["I'll have a glass of green smoothly in every morning.","I'll have a glass of green smoothy every morning."],["I felt energetic from this morning when I waked up, until now I feel like I can study more than usual.","I felt energetic this morning when I woke up, and now I feel like I can study more than usual."],["I wish Icould make tommorow active day!","I wish I could be more active tomorrow!"],["I wonder whether should I learn phonetics well first.","I wonder if I should learn phonetics first."],["When he first came to Japan, his Japanese classmates showed unpleasant faces when hearing his broken Japanese.","When he first came to Japan, his Japanese classmates made  unpleasant faces when hearing his broken Japanese."],["Also, many parents held the same altitude toward Chinese universities.","Also, many parents held the same attitude toward Chinese universities."],["Because I don't have diplom.","...I don't have a diploma."],["And I want to illustrate the articles with cartoon drawing.","And I want to illustrate the articles with cartoon drawings."],["What will I be able to eat???","I wonder what we'll be having for dinner?"],["But it is going to become a serious problem in near future.","But it is going to become a serious problem in the near future."],["Who can send man on the Mars in the next 15 years?","Who will send a man on  Mars in the next 15 years?"],["Here is a QUIZ for you.","OK, wish me good luck!"],["I didn't know what this sort of person want to do, so I had to keep Elain and the shopkeeper away from him.","I didn't know what this sort of person would want to do, so I had to keep Elain and the shopkeeper away from him."],["By \"Chae-bol hearing\" last week, we realized there scarcely separated collusive link between the entrepreneurs and the politicians from dictatorship in 1961.","At the chaebol hearing last week, we realized the scarcely  investigated collusive link between the entrepreneurs and the politicians from the dictatorship in 1961."],["I disagree with the either continuation or expansion of them without solving safety problems.","I disagree with the continuation or expansion of them without solving safety problems."],["Since then, I have not got a serious sick.","Since then, I have not had a serious attack."],["I've waited about 1 to 1 and a half hour for the lesson.","I've waited about an hour and a half for class to start."],["Why not?.","Why?"],["Felt little bit strange, and scared, too.","I felt little bit strange, and scared, too."],["I have many foreign student's friends and I always introduce about Japan so I'm used to introducing that.","I have many foreign students as my friends and I always introduce Japan, so I'm used to introducing that."],["However, he added, \"He might have corrected your sentence because you're Japanese.\"  I asked him what he meant, and he said, \"Because the Japanese are famous for not being at English.\"  I was shocked to hear that.","However, he added, \"He might have corrected your sentence because you're Japanese.\" I asked him what he meant, and he said, \"Because the Japanese are famous for not being good at English.\" I was shocked to hear that."],["Becoming the your weak point is it having your particular repeating that at everyday and every time.","You develop a weak point in that particular spot after repeating the same thing everyday."],["Both job are busy and hard for me,because my hotel are so big and busy,espesially winter season over 1000 people stay one night.","Both jobs are busy and hard for me because my hotel is so big and busy, especially in the winter season when over 1000 people stay in one night."],["Reluctantly,I leave that crying son passed to kindergarten teacher.","Reluctantly, I left that crying son and passed him to the kindergarten teacher."],["I have to try hard in few day until the that day come.","I have to try hard for a few days until the that day comes."],["They touch my soul and bring me freedom of spirit,the special feeling is hard to describe and I think it's difficult to find this feeling in daily life."," It touches my soul and ( bring me freedom of spirit ? ) <-- (end this with a \".\") This special feeling is hard to describe and I think  this feeling is difficult to find in daily life."],["Thank you for reading my entry.","Thank you for reading my entry. [Alternative: ... for reading this entry.]"],["I got this information from Wada san, she is a one of famous YouTuber.","I got this instruction from Wada san who is one of famous YouTuber."],["I went to a big shopping mall which is 25min drive from my house.","I went to a big shopping mall, which is 25 min away by car from my house."],["I ride motorbike to work every day.","I ride on motorbike to work every day."],["I have the best friends.","I have the best friends ever."],["I can talk to them at dining room in my school.","I can talk to them in the dining room in my school."],["I'm so happy.","I'm so happy"],["I almost forgot all the English(although I used to learn English is not very good).","I almost forgot all of my English (although my English was never very good)."],["Tomorrow is birthday to friend.","Tomorrow will be the birthday of my friend."],["Since starting these minor classes I have been put things in perspective.","Since starting these minor classes I have been putting things in perspective."],["I expect delicious meals.","I expect to eat delicious meals."],["In my opinion, advertisement can be understood as a sort of commercial dissemination expressed by artistic creativity.","In my opinion, advertisements can be understood as a sort of commercial dissemination, expressed by artistic creativity."],["How do they protect the iPhone from low temperature in the world?","How do they protect their iPhones from low temperatures elsewhere in the world?"],["Recently, there have been a lot of controversies about the idea that today's people have numerous information and will not need their parent's ideas.","Recently, there has been a lot of controversy about the idea that today's people have numerous information and will not need their parent's ideas."],["In my dream Iam a visitor to visit the west of China.","In my dream [I was a visitor to / I was visiting] the west of China."],["As a consequence, the combination of loss of money as well as credentials and lack of help from friends forces her to leave the country in advance.","As a consequence, the combination of the loss of money, as well as credentials, and a lack of help from friends, forced her to leave the country in advance."],["That was because all of you."," I was worried because of you."],["After the visit,her firend called one day to tell her that somthing is wrong with Ben.","After the visit, her friend called one day to tell her that something is wrong with Ben."],["I choose your university to earn doctorate because:","I choose your university to earn doctorate because:"],["Naruto is very popular in Japan, and I'm a big fan of it, too.","Naruto is very popular in Japan, and I'm a big fan of it, too."],["It was very slippery so I swept snow and sprinkled a little salt on the stairs outside.","It was very slippery, so I shoveled some snow and sprinkled a little salt on the stairs outside."],["The story about these man so impressed me!","The story about  this man impressed me so much!"],["They served excellent Japanese cuisine and our room commanded a fine view of the sea.","They served excellent Japanese cuisine and our room commanded a fine view of the sea."],["I take \"Karaage teishoku\" at restaurant.","I got \"Karaage teishoku\" at a restaurant."],["But there is a plenty of other lessons to try out.","But there are plenty of other lessons to try out."],["But they digged more size the hole in this room and they wanted to escape in the prison in next Friday.","But they dug more  holes in this room and they wanted to escape the prison next Friday."],["But when I saw RoboCop, I could concentrate it very well.","But when I watched RoboCop, I was able to concentrate really well."],["Nowadays I listen to foreign music, for example, Billy Joel, Eagles, and Nina.","Nowadays I listen to foreign music, such as, Billy Joel, Eagles, and Nina."],["Encouraged by my good friend, I tried to tell her in the new year of 2016.","Encouraged by my good friend, I tried to tell her in the new year of 2016."],["This app also reminds you of how many cups of water you have/haven't drunk every day.","This app also reminds you of how many cups of water you have/haven't drunk every day."],["We would run or hike more than 10km during around two hours.","We ran or hiked more than 10km [during a period of around / in about] two hours."]]
len(a)

500

In [106]:
def get_target_sents(args):
    
    # Targeted corpus
    metadata_params = construct_metadata_dict(args)
    corpus = get_by_metadata(CORPUS, **metadata_params)
    res = corpus
    if args["corpusType"] == 1:
        res = res[~res["agree"].isna()]
        # filtered by agreementType
        res = annotation_search(
            res,
            args["agreementType"],
            args["wordRange"],
            args["nCorrections"],
            args["sentenceType"],
        )
        return res
    elif args["corpusType"] == 0:
        res = res[res["agree"].isna()]
    else:
        res = res
    res = get_by_range(res, args["sentenceType"], args["wordRange"])
    res = get_by_multiple_corrections(res, args["nCorrections"])[["original", "corrected"]]
    return extract_pair_set(res)
    

In [110]:
sample = {
   "corpusType": 1,  #0-unannotated, 1-annotated, -1 -both (any)
   "agreementType": -1, #0 = 3 no's, 1 = 1 yes, 2 no's, 2 = 2 yeses 1 no, 3 = 3 yeses (if agreementType=-1, means any agreement type)
   "sentenceType": -1, #0-original, 1-corrected, -1 -both (any)
   "wordRange": -1, #based on 'sentenceType', sentences with word range in 10-20 inclusively (if wordRange = -1, means any word range)
   "nCorrections": -1, #sentences with n corrections (-1 means any)
   "sex": -1, #0-male, 1-female, 2-other (-1 means any)
   "occupation": -1, #should we have a list of fixed occupations or string is ok? ("all" means all :) )
   "location": -1, #should we have a list of fixed locations or string is ok? ("all" means all :) )
   "Lpoints": -1 #Lpoints in the given range inclusively ("all" means any)
}

In [113]:
list(get_target_sents(sample))

[('Well, you have to add some details, as globalization, increasing taxes, inmigration (salaries decreasing) and to get married is run a risk, because if you got divorced, very easy nowdays...',
  'Well, you have to add some details, as globalization, increasing taxes, immigration (salaries decreasing) and to get married is run a risk, because if you got divorced, very easy nowadays...'),
 ("It might be because, soon after Kenji's generation, Japan plunged into the era in which Japanese compelled Japanese to sacrifice themselves in insane way, for example Kamikaze or Banzai-Attack.",
  "It might be because soon after Kenji's generation, Japan plunged into the era in which the Japanese felt compelled to sacrifice themselves in insane ways, for example Kamikaze or Banzai-Attack."),
 ('People tend to go to foreign capital company when they take high score at the TOEIC test but it is ill-advised attempt because there are a lot of native English speakers and bilingual people.',
  'People te

In [98]:
pd.merge(CORPUS, METADATA, on="user_id").groupby(["location"]).count()["user_id"].sort_values(ascending=False).to_dict()

{'Japan': 17677,
 'China': 5354,
 'Taiwan': 1077,
 'Spain': 1061,
 'Russia': 1047,
 'Vietnam': 1047,
 'Korea': 1034,
 'Brazil': 905,
 'Belgium': 702,
 'U.S.A': 633,
 'France': 600,
 'Poland': 386,
 'Australia': 378,
 'Mexico': 357,
 'Indonesia': 279,
 'Italy': 228,
 'Germany': 218,
 'Panama': 160,
 'Hong Kong': 133,
 'Romania': 128,
 'United Kingdom': 116,
 'The Netherlands': 116,
 'Thailand': 112,
 'Colombia': 99,
 'Hungary': 94,
 'Ukraine': 91,
 'Norway': 76,
 'Jordan': 76,
 'Chile': 67,
 'India': 53,
 'Afghanistan': 48,
 'Iran': 45,
 'Peru': 43,
 'Canada': 41,
 'Venezuela': 39,
 'New Zealand': 29,
 'Saudi Arabia': 20,
 'Uruguay': 18,
 'Portugal': 17,
 'Bangladesh': 15,
 'Singapore': 13,
 'Egypt': 9,
 'Malaysia': 6,
 'Mongolia': 5,
 'Nicaragua': 4,
 'Cyprus': 3,
 'Latvia': 2,
 'Israel': 2,
 'Sri Lanka': 1,
 'Sweden': 1,
 'Ecuador': 1}